In [ ]:
import simdna
import simdna.synthetic as synthetic
import avutils
from avutils import util
reload(util)
import numpy as np
import momma_dragonn
reload(momma_dragonn)
import keras

In [ ]:
def generate_sequences_set(seq_length, num_seqs, motif_names):
    loadedMotifs = synthetic.LoadedEncodeMotifs(simdna.ENCODE_MOTIFS_PATH, pseudocountProb=0.001)
    embedInBackground = synthetic.EmbedInABackground(
        backgroundGenerator=synthetic.ZeroOrderBackgroundGenerator(seqLength=seq_length)
        , embedders=[
            synthetic.RepeatedEmbedder(
            synthetic.SubstringEmbedder(
                #synthetic.ReverseComplementWrapper(
                substringGenerator=synthetic.PwmSamplerFromLoadedMotifs(
                    loadedMotifs=loadedMotifs,motifName=motifName)
                #),
                ,positionGenerator=synthetic.UniformPositionGenerator()),
            quantityGenerator=#synthetic.FixedQuantityGenerator(1)
                synthetic.ZeroInflater(synthetic.MinMaxWrapper(
                synthetic.PoissonQuantityGenerator(1),
                theMax=1, theMin=0), zeroProb=0.0)
            )
            for motifName in motif_names
        ]
    )
    sequenceSetGenerator = synthetic.GenerateSequenceNTimes(embedInBackground, num_seqs)
    return sequenceSetGenerator

def one_hot_encode_sequences_set(sequence_set_generator):
    one_hot_encoded_sequences = []
    for sequence in sequence_set_generator.generateSequences():
        one_hot_encoded_sequences.append(avutils.util.seq_to_2d_image(sequence.seq))
    return np.array(one_hot_encoded_sequences)

seq_length=200


motif_names = ["CTCF_known1", "IRF_known1", "GATA_disc1"]

num_train_samples = 20000

one_hot_data_train_pos, one_hot_data_train_neg =\
 [one_hot_encode_sequences_set(generate_sequences_set(seq_length=seq_length, num_seqs=num_train_samples, motif_names=names))
  for names in [motif_names, []]]
one_hot_data_train_inputs = np.concatenate([one_hot_data_train_pos, one_hot_data_train_neg], axis=0)
one_hot_data_train_labels = np.concatenate([one_hot_data_train_pos, -one_hot_data_train_neg], axis=0)
util.shuffle_arrays([one_hot_data_train_inputs, one_hot_data_train_labels], copy_on_shuffle=True)

In [ ]:
reload(util)
one_hot_data_valid_pos, one_hot_data_valid_neg =\
 [one_hot_encode_sequences_set(generate_sequences_set(seq_length=seq_length, num_seqs=1000, motif_names=names))
  for names in [motif_names, []]]
one_hot_data_valid_inputs = np.concatenate([one_hot_data_valid_pos, one_hot_data_valid_neg], axis=0)
one_hot_data_valid_labels = np.concatenate([one_hot_data_valid_pos, -one_hot_data_valid_neg], axis=0)
util.shuffle_arrays([one_hot_data_valid_inputs, one_hot_data_valid_labels], copy_on_shuffle=True)

In [ ]:
import keras
reload(keras)
import keras.objectives
reload(keras.objectives)
from keras import backend
reload(backend)
import keras.layers.convolutional
reload(keras.layers.convolutional)
import momma_dragonn
reload(momma_dragonn)
reload(momma_dragonn.end_of_epoch_callbacks)
reload(momma_dragonn.data_loaders)
reload(momma_dragonn.model_evaluators)
reload(momma_dragonn.model_trainers.keras_model_trainer)
reload(momma_dragonn.data_loaders.core)

def model_creator_func(nb_filter):
    filter_width=20
    #maxpool_filter_width=seq_length-(filter_width-1) #pool over entire region for now
    maxpool_filter_width=20
    
    from keras.models import Graph
    graph = Graph() 
    graph.add_input(name="sequence", input_shape=(1,4,seq_length))
    #add convolutional layer
    graph.add_node(
        keras.layers.convolutional.Convolution2D(nb_filter=nb_filter, nb_row=4, nb_col=filter_width),
        name="conv", input="sequence")
    #add maxpool filter layer
    graph.add_node(
        keras.layers.convolutional.MaxPoolFilter2D_CenteredPool(pool_size=(1,maxpool_filter_width)),
        #keras.layers.convolutional.MaxPoolFilter2D_NonOverlapStrides(pool_size=(1,maxpool_filter_width)),
        name="filt", input="conv")
    #add a padding layer so deconv will be the right size
    graph.add_node(
        keras.layers.convolutional.ZeroPadding2D(padding=(0,filter_width-1)),
        name="padding", input="filt")
    #add a deconv layer to reconstruct the input
    graph.add_node(
        keras.layers.convolutional.Convolution2D(
            nb_filter=4, nb_row=1, nb_col=filter_width,
            W_constraint=keras.constraints.MaxNorm(m=10)),
        name="deconv", input="padding")
    #transpose to make the deconv axis the row axis
    graph.add_node(
        keras.layers.convolutional.ExchangeChannelsAndRows(),
        name="swapaxes", input="deconv")
    #softmax across rows
    graph.add_node(
        keras.layers.convolutional.SoftmaxAcrossRows(),
        name="output_softmax", input="swapaxes")
    #designate output node
    graph.add_output(name="output", input="output_softmax")
    #compile
    graph.compile(
        optimizer=keras.optimizers.Adam(),
        loss={"output": "one_hot_rows_categorical_cross_entropy"}
    )
    return graph

#model creator
model_creator = momma_dragonn.model_creators.flexible_keras.KerasModelFromFunc(
    func=lambda: model_creator_func(20*len(motif_names)),
    model_wrapper_class=momma_dragonn.model_wrappers.keras_model_wrappers.KerasGraphModelWrapper)    

#data loaders
train_data_loader = momma_dragonn.data_loaders.core.BatchDataLoader_XYDictAPI(
                        X={'sequence': one_hot_data_train_inputs}, Y={'output': one_hot_data_train_labels},
                        weight={}, batch_size=200, num_to_load_for_eval=num_train_samples, bundle_x_and_y_in_generator=True)
valid_data_loader = momma_dragonn.data_loaders.core.AtOnceDataLoader_XYDictAPI(
                        X={'sequence': one_hot_data_valid_inputs}, Y={'output': one_hot_data_valid_labels})
#model evaluator
model_evaluator = momma_dragonn.model_evaluators.GraphAccuracyStats(
    key_metric="onehot_rows_crossent", all_metrics=["onehot_rows_crossent"])

#stopping criterion
stopping_criterion_config = {"class": "EarlyStopping", "kwargs": {"max_epochs": 300, "epochs_to_wait": 3}}

#callbacks
end_of_epoch_callbacks = []#momma_dragonn.end_of_epoch_callbacks.PrintPerfAfterEpoch(print_trend=True)]

#trainer
trainer = momma_dragonn.model_trainers.keras_model_trainer.KerasFitGeneratorModelTrainer(
    samples_per_epoch=num_train_samples, stopping_criterion_config=stopping_criterion_config)

#train model
model_wrapper, performance_history, training_metadata = trainer.train(model_wrapper=model_creator.get_model_wrapper(),
                                                          model_evaluator=model_evaluator,
                                                          valid_data_loader=valid_data_loader,
                                                          other_data_loaders={'train': train_data_loader},
                                                          end_of_epoch_callbacks=end_of_epoch_callbacks)

In [1]:
file_prefix="twohundredbpseq_3motifs_50pcchance_centeredpool"

In [ ]:
model_wrapper.create_files_to_save(directory=".", prefix=file_prefix)

In [2]:
from keras.models import model_from_yaml                                    
model = model_from_yaml(open(file_prefix+"_modelYaml.yaml").read())
model.load_weights(file_prefix+"_modelWeights.h5")

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5005)
/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [3]:
deconv_weights, deconv_biases = model.nodes['deconv'].get_weights()

In [4]:
deconv_weights

array([[[[ 1.64436805,  1.43902874,  1.39167547, ...,  1.08905792,
           0.7203052 ,  0.6786744 ]],

        [[ 0.67296028,  0.28336164,  1.08435118, ...,  0.47243041,
           0.32687724,  0.86565262]],

        [[-2.02874017, -1.71229529, -1.62211907, ..., -0.49180073,
          -0.5963093 , -0.47461241]],

        ..., 
        [[ 0.7144652 ,  1.24396169,  1.06769347, ...,  0.62082899,
           1.40083122,  1.38282824]],

        [[ 0.5726794 ,  0.64030915,  0.31980035, ...,  0.90434176,
           0.86987448,  1.23142123]],

        [[ 1.24721742, -0.00546391,  0.91923374, ...,  1.42352414,
           1.00433457,  0.97501218]]],


       [[[-1.14137864, -1.14972091, -1.09991622, ..., -0.54313135,
          -0.13502759, -0.61139762]],

        [[-0.54975289, -0.40143117, -0.62494916, ..., -0.47975978,
          -0.58545816, -0.01525566]],

        [[ 1.37923717,  1.30462456,  1.19325984, ...,  0.31496641,
           0.28718817,  0.34297657]],

        ..., 
        [[-0.345

In [5]:
deconv_biases

array([ 1.98738754, -1.90291011, -1.1131053 , -2.54785991], dtype=float32)

In [6]:
#import deepLIFT stuff
import os, sys
import numpy as np
scriptsDir = os.environ.get("ENHANCER_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable ENHANCER_SCRIPTS_DIR to point to enhancer_prediction_code");
sys.path.insert(0,scriptsDir+"/featureSelector/deepLIFFT/");
import criticalSubsetIdentification as csi
import deepLIFTutils
reload(deepLIFTutils)

<module 'deepLIFTutils' from '/Users/avantishrikumar/Research/Enhancer_Prediction/enhancer_prediction_code/featureSelector/deepLIFFT/deepLIFTutils.pyc'>

In [7]:
weights = np.transpose(deconv_weights,(1,2,0,3))
weights = weights[:,:,:,::-1]
print(weights.shape)
filterScores = np.sum((np.max(weights,axis=-1)-np.min(weights,axis=-1)),axis=-1)
sortedFilters = sorted(enumerate(filterScores), key=lambda x: -x[1])
print(sortedFilters)

(60, 1, 4, 20)
[(3, array([ 10.2397213], dtype=float32)), (20, array([ 9.47280407], dtype=float32)), (10, array([ 8.93774223], dtype=float32)), (17, array([ 8.79210472], dtype=float32)), (26, array([ 8.61362457], dtype=float32)), (7, array([ 8.3150053], dtype=float32)), (0, array([ 8.22377777], dtype=float32)), (59, array([ 8.11780071], dtype=float32)), (25, array([ 7.73480701], dtype=float32)), (22, array([ 7.49399996], dtype=float32)), (55, array([ 7.44176292], dtype=float32)), (12, array([ 7.40381241], dtype=float32)), (30, array([ 7.33690977], dtype=float32)), (40, array([ 7.26463032], dtype=float32)), (23, array([ 7.21216297], dtype=float32)), (32, array([ 6.97490311], dtype=float32)), (37, array([ 6.90068531], dtype=float32)), (2, array([ 6.86307907], dtype=float32)), (16, array([ 6.74383831], dtype=float32)), (45, array([ 6.68700027], dtype=float32)), (9, array([ 6.63256645], dtype=float32)), (14, array([ 6.49044514], dtype=float32)), (6, array([ 6.36226511], dtype=float32)), (3

In [ ]:
deepLIFTutils.printConvolution(weights, np.zeros(len(weights)), 3, addToTitle="", printBias=False, figSize=None, outputFile=None)

In [ ]:
deepLIFTutils.plotPositiveAndNegativeOutlierFilters(sortedFilters=sortedFilters
                                                    , numPositiveOutliers=5
                                                    , numNegativeOutliers=1
                                                    , weights=weights
                                                    , biases=np.zeros(len(weights))
                                                    , printBias=False
                                                    , figSize=(10,2))